In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

In [2]:
tf.config.list_physical_devices('GPU')

[]

# ResNet 기본 블록

In [ ]:
def conv_block(filters=64, kernel_size=3, )

In [4]:
# function for building ResNet Block

def build_resnet34_block(input_layer,
                         num_cnn=3, 
                         channel=64,
                         block_num=1,
                      ):
    # 입력 레이어
    x = input_layer

    # CNN 레이어
    for cnn_num in range(num_cnn):
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}'
        )(x)   

    return x

In [ ]:
# function for building ResNet Block

def build_resnet50_block(input_layer,
                         num_cnn=3, 
                         channel=64,
                         block_num=1,
                      ):
    # 입력 레이어
    x = input_layer

    # CNN 레이어
    for cnn_num in range(num_cnn):
        x = keras.layers.Conv2D(
            filters=channel,
            kernel_size=(3,3),
            activation='relu',
            kernel_initializer='he_normal',
            padding='same',
            name=f'block{block_num}_conv{cnn_num}'
        )(x)

    return x

# Model implementation

In [ ]:
# VGG 모델 자체를 생성하는 함수입니다.
def build_resnet(input_shape=(32,32,3),
                 num_cnn_list=[3,4,6,3],
                 channel_list=[64,128,256,512],
                 num_classes=10
                 is_50=False
                ):
    
    assert len(num_cnn_list) == len(channel_list) #모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    output = input_layer
    
    # 7x7 convolution
    output = keras.layers.Conv2D(
        filters=64,
        kernel_size=(7,7),
        strides=(2,2),
        kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01),
        padding='same'
    )(output)
    output = keras.layers.BatchNormalization()(output)
    output = keras.layers.Activation('relu')(output)
    
    # maxpooling2D
    output = keras.layers.MaxPooling2D(
        pool_size=(2, 2),
        strides=2,
        name='stage_2_0_pooling'
    )(output)
    
    
    # config list들의 길이만큼 반복해서 블록을 생성합니다.
    if is_50: # ResNet-50
        for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
            output = build_resnet50_block(
                output,
                num_cnn=num_cnn, 
                channel=channel,
                block_num=i
            )
    else: # ResNet-34
        for i, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
            output = build_resnet34_block(
                output,
                num_cnn=num_cnn, 
                channel=channel,
                block_num=i
            )
        
    
    output = keras.layers.AveragePooling2D(pool_size=1)(output)
    output = keras.layers.Flatten(name='flatten')(output)
    output = keras.layers.Dense(num_classes, activation='softmax', name='fc10')(output)
    
    model = keras.Model(
        inputs=input_layer, 
        outputs=output
    )
    return model